MNIST Datasets | Training < 20 epochs | Parameters < 20k | Accuracy > 99.4

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, 3), # 26
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1),

            nn.Conv2d(16, 32, 3), # 24
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.1),

            nn.Conv2d(32, 10, 1),
            nn.ReLU(),

            nn.MaxPool2d(2, 2), # 12
            
            nn.Conv2d(10, 32, 3), # 10
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.1),
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 10, 1),
            nn.ReLU(),

            nn.Conv2d(10, 32, 3), # 8
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.1),

            nn.Conv2d(32, 10, 1), # 6
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout(0.1),

            nn.Conv2d(10, 16, 3),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1),

        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(16, 10, 1),
            nn.ReLU(),
            nn.Conv2d(10, 10, 6),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout(0.1),
        )
           
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = x.view(x.size(0), -1)
        x = F.log_softmax(x, dim=1)
        return x 

       

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
           Dropout-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 32, 24, 24]           4,640
              ReLU-6           [-1, 32, 24, 24]               0
       BatchNorm2d-7           [-1, 32, 24, 24]              64
           Dropout-8           [-1, 32, 24, 24]               0
            Conv2d-9           [-1, 10, 24, 24]             330
             ReLU-10           [-1, 10, 24, 24]               0
        MaxPool2d-11           [-1, 10, 12, 12]               0
           Conv2d-12           [-1, 32, 10, 10]           2,912
             ReLU-13           [-1, 32, 10, 10]               0
      BatchNorm2d-14           [-1, 32,

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
lre = 0.01
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lre, momentum=0.9)

for epoch in range(1, 21):
    print(f"Epoch: {epoch}, Learning rate: {lre}")
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    if epoch > 10:
      lre = round(0.003 * 1/(1 + 0.319 * epoch), 10)
      optimizer = optim.SGD(model.parameters(), lr=lre, momentum=0.9)

Epoch: 1, Learning rate: 0.01


loss=0.3528384268283844 batch_id=468: 100%|██████████| 469/469 [00:24<00:00, 19.25it/s]



Test set: Average loss: 0.1047, Accuracy: 9753/10000 (97.5300%)

Epoch: 2, Learning rate: 0.01


loss=0.2628697454929352 batch_id=468: 100%|██████████| 469/469 [00:24<00:00, 19.08it/s]



Test set: Average loss: 0.0668, Accuracy: 9825/10000 (98.2500%)

Epoch: 3, Learning rate: 0.01


...


Test set: Average loss: 0.0219, Accuracy: 9942/10000 (99.4200%)

Epoch: 19, Learning rate: 0.0004449718


loss=0.10677972435951233 batch_id=468: 100%|██████████| 469/469 [00:25<00:00, 18.74it/s]



Test set: Average loss: 0.0205, Accuracy: 9942/10000 (99.4200%)

Epoch: 20, Learning rate: 0.000424869


loss=0.09717757254838943 batch_id=468: 100%|██████████| 469/469 [00:24<00:00, 18.97it/s]



Test set: Average loss: 0.0212, Accuracy: 9942/10000 (99.4200%)

